In [6]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
import importlib

from utils.paths import SetupPaths
import utils.readsubfHDF5Py3 as readSub
from utils.read_group_cats import ReadCats
from utils.merger_trees import TraceMergerTree
from utils.vectorCorrection import vectorCorrection as vector

paths = SetupPaths()

In [10]:
catpath = paths.path_tnghydro
snap = 99
keysel_default =  [
            'GroupPos','Group_M_TopHat200', 
            'Group_R_TopHat200','GroupNsubs',
            'GroupFirstSub',
            'SubhaloGrNr','SubhaloMass',
            'SubhaloPos','SubhaloVel',
            'SubhaloMassType']
little_h = 0.6774

catalog = readSub.subfind_catalog(
            basedir=catpath,
            snapnum=snap,keysel=keysel_default)

In [ ]:
print(catalog.Sub)
print(catalog.Group_M_TopHat200[group])
print(catalog.GroupNsubs[group])

In [17]:
fullcat_stars = catalog.SubhaloMassType[:,4]/little_h 

In [43]:
def get_primmask(primstells, size):
    if size == "dwarf":
        mask = (primstells > 0.01) & (primstells < 0.5)
    elif size == "massive":
        mask = (primstells > 0.5) & (primstells < 10)
    return mask

def get_groupmask(groupmass, size):
    if size == "dwarf":
        mask = (groupmass > 8) & (groupmass < 50)
    elif size == "massive":
        mask = (groupmass > 100) & (groupmass < 650)
    return mask

In [20]:
fullcat_mask = get_primmask(fullcat_stars,"massive")

In [23]:
fullcat_passcount = np.count_nonzero(fullcat_mask)
fullcat_passcount

8987

In [35]:
f.keys()

<KeysViewHDF5 ['Header', 'pairs', 'unpaired']>

In [39]:
f = h5py.File(f'{paths.path_pairs}TNG_99_1000.hdf5')

paircat = {}
unpaircat = {}
for key, val in f['pairs']['hydro'].items():
    paircat[key] = np.array(val)
for key, val in f['unpaired']['hydro'].items():
    unpaircat[key] = np.array(val)    
f.close()

In [46]:
pair_realmask = paircat['Realization']==0
unpair_realmask = unpaircat['Realization']==0

In [41]:
unpaircat.keys()

dict_keys(['Group ID', 'Group Mass', 'Group Nsubs', 'Group Radius', 'Realization', 'Sub1 BHMass', 'Sub1 BHMdot', 'Sub1 GasMetallicity', 'Sub1 ID', 'Sub1 Mass', 'Sub1 MassType', 'Sub1 Pos', 'Sub1 SFR', 'Sub1 SFRinRad', 'Sub1 Stellar Mass', 'Sub1 Vel'])

In [69]:
## get all unpaired primaries
unpair_stell = get_primmask(unpaircat['Sub1 Stellar Mass'], "massive")
unpair_group = get_groupmask(unpaircat['Group Mass'], "massive")                
print(np.count_nonzero(unpair_stell))
print(np.count_nonzero(unpair_group))

## get all paired primaries 
pair_stell = get_primmask(paircat['Sub1 Stellar Mass'], "massive")
pair_group = get_groupmask(paircat['Group Mass'], "massive")
print(np.count_nonzero(pair_stell))
print(np.count_nonzero(pair_group))

unpair_prims = unpair_stell & unpair_group & unpair_realmask
pair_prims = pair_stell & pair_group & pair_realmask
print(np.count_nonzero(unpair_prims))
print(np.count_nonzero(pair_prims))
paircat_passcount = np.count_nonzero(unpair_prims)+np.count_nonzero(pair_prims)

581473
0
7345591
1855854
0
1800


In [48]:
## all primaries in one realization
unpaircat['Sub1 Stellar Mass'][unpair_prims]

array([], dtype=float64)

identifying the % of my sample in the full volume sample

In [54]:
len(fullcat_mask)

4371211

In [58]:
fullcat_subIDs = np.arange(0,len(fullcat_mask),1)[fullcat_mask]

In [63]:
paircat_subIDs = paircat['Sub1 ID'][pair_prims]

In [67]:
overlap = np.count_nonzero(np.isin(paircat_subIDs, fullcat_subIDs))

# Summary

In [70]:
print(f'Number of subhalos in full volume that pass simulation stellar mass criteria: {fullcat_passcount}')
print(f'Number of primary subhalos in my sample: {paircat_passcount}')
print(f'Number of sample primaries in full volume selection: {overlap}')

print(f'Ratio of number in my sample to full volume selection: {paircat_passcount/fullcat_passcount}')
print(f'Ratio of my sample captured from full volume selection: {overlap/paircat_passcount}')
print(f'Ratio of my sample captured compared to full volume: {overlap/fullcat_passcount}')



Number of subhalos in full volume that pass simulation stellar mass criteria: 8987
Number of primary subhalos in my sample: 1800
Number of sample primaries in full volume selection: 1659
Ratio of number in my sample to full volume selection: 0.20028930677645487
Ratio of my sample captured from full volume selection: 0.9216666666666666
Ratio of my sample captured compared to full volume: 0.18459997774563258


In [73]:
def get_summary_stats(snapshot, size):
    catpath = paths.path_tnghydro
    snap = snapshot
    little_h = 0.6774
    keysel_default =  [
                'GroupPos','Group_M_TopHat200', 
                'Group_R_TopHat200','GroupNsubs',
                'GroupFirstSub',
                'SubhaloGrNr','SubhaloMass',
                'SubhaloPos','SubhaloVel',
                'SubhaloMassType']

    catalog = readSub.subfind_catalog(
                basedir=catpath,
                snapnum=snap,keysel=keysel_default)
    fullcat_stars = catalog.SubhaloMassType[:,4]/little_h 
    fullcat_mask = get_primmask(fullcat_stars,size)
    fullcat_passcount = np.count_nonzero(fullcat_mask)

    ## unpacking data
    f = h5py.File(f'{paths.path_pairs}TNG_{snapshot}_10.hdf5')
    paircat = {}
    unpaircat = {}
    for key, val in f['pairs']['hydro'].items():
        paircat[key] = np.array(val)
    for key, val in f['unpaired']['hydro'].items():
        unpaircat[key] = np.array(val)    
    f.close()
    
    pair_realmask = paircat['Realization']==0
    unpair_realmask = unpaircat['Realization']==0
    
    ## get all unpaired primaries
    unpair_stell = get_primmask(unpaircat['Sub1 Stellar Mass'], size)
    unpair_group = get_groupmask(unpaircat['Group Mass'], size)                

    ## get all paired primaries 
    pair_stell = get_primmask(paircat['Sub1 Stellar Mass'], size)
    pair_group = get_groupmask(paircat['Group Mass'], size)

    unpair_prims = unpair_stell & unpair_group & unpair_realmask
    pair_prims = pair_stell & pair_group & pair_realmask
    paircat_passcount = np.count_nonzero(unpair_prims)+np.count_nonzero(pair_prims)
    
    fullcat_subIDs = np.arange(0,len(fullcat_mask),1)[fullcat_mask]
    paircat_subIDs = paircat['Sub1 ID'][pair_prims]
    overlap = np.count_nonzero(np.isin(paircat_subIDs, fullcat_subIDs))
    
    print(f'Number of subhalos in full volume that pass simulation stellar mass criteria: {fullcat_passcount}')
    print(f'Number of primary subhalos in my sample: {paircat_passcount}')
    print(f'Number of sample primaries in full volume selection: {overlap}')

    print(f'Ratio of number in my sample to full volume selection: {paircat_passcount/fullcat_passcount}')
    print(f'Ratio of my sample captured from full volume selection: {overlap/paircat_passcount}')
    print(f'Ratio of my sample captured compared to full volume: {overlap/fullcat_passcount}')
    
    return fullcat_passcount, paircat_passcount, overlap



In [74]:
get_summary_stats(99,"massive")

Number of subhalos in full volume that pass simulation stellar mass criteria: 8987
Number of primary subhalos in my sample: 1800
Number of sample primaries in full volume selection: 1659
Ratio of number in my sample to full volume selection: 0.20028930677645487
Ratio of my sample captured from full volume selection: 0.9216666666666666
Ratio of my sample captured compared to full volume: 0.18459997774563258


(8987, 1800, 1659)

In [75]:
get_summary_stats(99,"dwarf")

Number of subhalos in full volume that pass simulation stellar mass criteria: 44944
Number of primary subhalos in my sample: 11982
Number of sample primaries in full volume selection: 7503
Ratio of number in my sample to full volume selection: 0.26659843360626556
Ratio of my sample captured from full volume selection: 0.6261892839258888
Ratio of my sample captured compared to full volume: 0.16694108223567106


(44944, 11982, 7503)

In [76]:
get_summary_stats(49,"dwarf")

Number of subhalos in full volume that pass simulation stellar mass criteria: 47050
Number of primary subhalos in my sample: 15344
Number of sample primaries in full volume selection: 11425
Ratio of number in my sample to full volume selection: 0.3261211477151966
Ratio of my sample captured from full volume selection: 0.7445907194994786
Ratio of my sample captured compared to full volume: 0.242826780021254


(47050, 15344, 11425)

In [77]:
get_summary_stats(49,"massive")

Number of subhalos in full volume that pass simulation stellar mass criteria: 7479
Number of primary subhalos in my sample: 1756
Number of sample primaries in full volume selection: 1655
Ratio of number in my sample to full volume selection: 0.2347907474261265
Ratio of my sample captured from full volume selection: 0.9424829157175398
Ratio of my sample captured compared to full volume: 0.22128626821767616


(7479, 1756, 1655)